In [ ]:
!pip install -q kaggle

In [ ]:
import os
from google.colab import files
uploaded = files.upload()
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

# Print the current working directory
print("\nCurrent working directory:")
print(os.getcwd())

# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Saving kaggle.json to kaggle.json
User uploaded file "kaggle.json" with length 69 bytes

Current working directory:
/content


In [ ]:
import kaggle
import os

# 1. Download a Dataset:

# Replace 'dataset_name' with the actual name of the dataset.
# The format is usually 'username/dataset-name'.  Find this on the Kaggle dataset page's URL.
dataset_name = 'paultimothymooney/stock-market-data'  # Example: 'uciml/iris'

# Option 1: Download to current directory:
kaggle.api.dataset_download_files(dataset_name, path='.', unzip=True)  # Unzip the data

# Option 2: Download to a specific directory (recommended):
download_path = 'data/'  # Create a directory if it doesn't exist
if not os.path.exists(download_path):
    os.makedirs(download_path)
kaggle.api.dataset_download_files(dataset_name, path=download_path, unzip=True)

# 2. Download a Competition Dataset:

# competition_name = 'name-of-competition'
# kaggle.api.competition_download_files(competition_name, path='.', unzip=True)

# 3. List Datasets (for finding dataset names):
# datasets = kaggle.api.dataset_list(search='keyword') # Replace keyword with a search term.
# for dataset in datasets:
#     print(dataset)

# After downloading, you can use pandas to read the data:


Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/stock-market-data
Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/stock-market-data


In [ ]:
import pandas as pd

# Example (adjust filename as needed, and adjust path if needed)
try:
   stock_df = pd.read_csv(os.path.join(download_path, '/content/stock_market_data/forbes2000/csv/AAPL.csv')) # Replace 'your_data_file.csv' and path if needed
   print(stock_df.head())
except FileNotFoundError:
   print("Data file not found.  Check the path and filename.")


stock_df.head()


         Date       Low      Open     Volume      High     Close  \
0  12-12-1980  0.128348  0.128348  469033600  0.128906  0.128348   
1  15-12-1980  0.121652  0.122210  175884800  0.122210  0.121652   
2  16-12-1980  0.112723  0.113281  105728000  0.113281  0.112723   
3  17-12-1980  0.115513  0.115513   86441600  0.116071  0.115513   
4  18-12-1980  0.118862  0.118862   73449600  0.119420  0.118862   

   Adjusted Close  
0        0.099874  
1        0.094663  
2        0.087715  
3        0.089886  
4        0.092492  


,Date,Low,Open,Volume,High,Close,Adjusted Close
0,12-12-1980,0.128348,0.128348,469033600,0.128906,0.128348,0.099874
1,15-12-1980,0.121652,0.122210,175884800,0.122210,0.121652,0.094663
2,16-12-1980,0.112723,0.113281,105728000,0.113281,0.112723,0.087715
3,17-12-1980,0.115513,0.115513,86441600,0.116071,0.115513,0.089886
4,18-12-1980,0.118862,0.118862,73449600,0.119420,0.118862,0.092492


In [ ]:
import requests
import pandas as pd

api_key = "BG644YLMMDUFXHCB"

def get_earnings_data(ticker):
    """Fetch earnings report dates from Alpha Vantage"""
    url = f"https://www.alphavantage.co/query?function=EARNINGS&symbol={ticker}&apikey={api_key}"
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data for {ticker}: {e}")
        return pd.DataFrame()  # Return an empty DataFrame on error

    earnings_list = []

    if "quarterlyEarnings" in data:
        for entry in data["quarterlyEarnings"]:
            try:
                earnings_list.append({
                    "symbol": ticker,
                    "reported_date": pd.to_datetime(entry["reportedDate"]),
                    "actual_eps": float(entry["reportedEPS"]) if entry["reportedEPS"] != "None" else None,
                    "estimated_eps": float(entry["estimatedEPS"]) if entry["estimatedEPS"] != "None" else None,
                    "surprise": float(entry["surprise"]) if entry["surprise"] != "None" else None,
                    "surprise_pct": float(entry["surprisePercentage"]) if entry["surprisePercentage"] != "None" else None,
                })
            except KeyError as e:
                print(f"KeyError in earnings data for {ticker}: {e}")
                continue  # Skip this entry and continue with the next

    print(f"Loaded {len(earnings_list)} earnings reports for {ticker}")
    # print(f"Earning Reports: {earnings_list}")  # Remove this line for production

    return pd.DataFrame(earnings_list)


earnings_data = get_earnings_data('AAPL')
earnings_data.head()

Loaded 116 earnings reports for AAPL


,symbol,reported_date,actual_eps,estimated_eps,surprise,surprise_pct
0,AAPL,2025-01-30,2.40,2.36,0.04,1.6949
1,AAPL,2024-10-31,1.64,1.60,0.04,2.5000
2,AAPL,2024-08-01,1.40,1.35,0.05,3.7037
3,AAPL,2024-05-02,1.53,1.50,0.03,2.0000
4,AAPL,2024-02-01,2.18,2.10,0.08,3.8095


In [ ]:
import os
import numpy as np
import pandas as pd

def add_ticker_column(df, filename, index):
    """Adds a ticker column to the DataFrame based on the filename."""
    ticker = filename.split('.')[0]  # Extract ticker from filename
    df['Ticker'] = ticker
    df['Index'] = index
    return df

def load_and_combine_csv_data(base_dir):
    """
    Loads all CSV files from all subdirectories in a base directory, adds a Ticker column,
    and combines them into a single DataFrame.
    """
    all_data = []
    for subdir in os.listdir(base_dir): # go through base, where stock market dir is
        subdir_path = os.path.join(base_dir, subdir)
        if os.path.isdir(subdir_path): # If is dir

            csv_dir = os.path.join(subdir_path, "csv") # go to the csv dir inside the base
            if os.path.exists(csv_dir):
                for filename in os.listdir(csv_dir):
                    if filename.endswith(".csv"):
                        csv_path = os.path.join(csv_dir, filename)
                        try:
                            df = pd.read_csv(csv_path, parse_dates=['Date']) # Parse date

                            # Date is not defined, print so we know
                            if 'Date' not in df:
                                print(f"Issue, Date column not defined in {filename}")


                            df = add_ticker_column(df, filename, subdir_path) # ticker to df from file name
                            all_data.append(df) # Append the df to all_data

                        except Exception as e:
                            print(f"Error loading or processing {filename}: {e}")
            else:
                print(f"csv directory not found in {subdir}")

    combined_df = pd.concat(all_data, ignore_index=True) # combine
    return combined_df

# --- Main ---
base_dir = "/content/stock_market_data/"  # The main directory with the four folders
combined_stock_data = load_and_combine_csv_data(base_dir) # Loads in all the data to a data frame
print(f"Combined data shape: {combined_stock_data.shape}")
print(f"Combined data columns: {combined_stock_data.columns}")
print(combined_stock_data.head()) # Print the head of the data

In [ ]:
print(f"Combined data shape: {combined_stock_data.shape}")
print(f"Combined data columns: {combined_stock_data.columns}")

# Print unique tickers
unique_tickers = combined_stock_data['Ticker'].unique() # Get unique values

# print("Unique Tickers:")
# for ticker in unique_tickers: # Print
#     print(ticker)

combined_stock_data.head()

Combined data shape: (24604971, 8)
Combined data columns: Index(['Low', 'Open', 'Volume', 'High', 'Close', 'Adjusted Close', 'Ticker',
       'Index'],
      dtype='object')


,Low,Open,Volume,High,Close,Adjusted Close,Ticker,Index
Date,,,,,,,,
07-07-1994,9.0000,9.0000,572400.0,9.3750,9.3125,3.022412,AKO-A,/content/stock_market_data/nyse
08-07-1994,9.1875,9.4375,109200.0,9.4375,9.1875,2.981842,AKO-A,/content/stock_market_data/nyse
11-07-1994,9.0000,9.1250,149200.0,9.1250,9.0000,2.920988,AKO-A,/content/stock_market_data/nyse
12-07-1994,9.0000,9.1250,72400.0,9.1250,9.1250,2.961556,AKO-A,/content/stock_market_data/nyse
13-07-1994,9.0625,9.1250,192000.0,9.1875,9.1875,2.981842,AKO-A,/content/stock_market_data/nyse


In [1]:
# Export to CSV
output_csv_path = "/content/combined_stock_data.csv"  # Choose where to save the combined CSV
combined_stock_data.to_csv(output_csv_path, index=False)  # Export to CSV, don't include index

print(f"Combined data exported to: {output_csv_path}")

NameError: name 'combined_stock_data' is not defined

In [5]:
# Sort the data by 'Ticker' and 'Date'
combined_stock_data = combined_stock_data.sort_values(['Ticker', 'Date'])

# Group by 'Ticker' and shift the 'Open' price
combined_stock_data['Next_Day_Open'] = combined_stock_data.groupby('Ticker')['Open'].shift(-1)

NameError: name 'combined_stock_data' is not defined

Upload file from local

In [6]:
import pandas as pd
import os

# Load CSV, convert Date, populate Next_Day_Open (as in the previous complete example)
# (Paste the complete code from my previous answer here - file loading, date conversion, Next_Day_Open calculation)
# Replace 'your_data.csv' with your actual filename

#Load your CSV data (replace 'your_data.csv' with your actual filename)
file_path = 'combined_stock_data.csv'  # Replace with your actual filename
if os.path.exists(file_path):
    try:
        combined_df = pd.read_csv(file_path, index_col=0)
    except FileNotFoundError:
        print(f"Error: File '{file_path}' not found. Double-check the path.")
    except pd.errors.EmptyDataError:
        print(f"Error: File '{file_path}' is empty.")
    except pd.errors.ParserError:
        print(f"Error: Problem parsing '{file_path}'. Check for incorrect delimiters or data types.")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
else:
   print(f"Error: File '{file_path}' not found. Make sure the filename and path are correct.")
   combined_df = None #ensure combined_df is None if no data loaded



In [14]:
combined_df.head()

combined_stock_data = combined_df.copy()

combined_stock_data.head()


,Low,Open,Volume,High,Close,Adjusted Close,Ticker,Index
Date,,,,,,,,
07-07-1994,9.0000,9.0000,572400.0,9.3750,9.3125,3.022412,AKO-A,/content/stock_market_data/nyse
08-07-1994,9.1875,9.4375,109200.0,9.4375,9.1875,2.981842,AKO-A,/content/stock_market_data/nyse
11-07-1994,9.0000,9.1250,149200.0,9.1250,9.0000,2.920988,AKO-A,/content/stock_market_data/nyse
12-07-1994,9.0000,9.1250,72400.0,9.1250,9.1250,2.961556,AKO-A,/content/stock_market_data/nyse
13-07-1994,9.0625,9.1250,192000.0,9.1875,9.1875,2.981842,AKO-A,/content/stock_market_data/nyse


In [18]:
combined_stock_data.columns

print(combined_stock_data.index)

Index(['07-07-1994', '08-07-1994', '11-07-1994', '12-07-1994', '13-07-1994',
       '14-07-1994', '15-07-1994', '18-07-1994', '19-07-1994', '20-07-1994',
       ...
       '29-11-2022', '30-11-2022', '01-12-2022', '02-12-2022', '05-12-2022',
       '06-12-2022', '07-12-2022', '08-12-2022', '09-12-2022', '12-12-2022'],
      dtype='object', name='Date', length=24604971)


In [21]:
import pandas as pd

# Assuming 'combined_stock_data' DataFrame is already loaded
if 'combined_stock_data' in locals():
    # Reset the index, making 'Date' a column again
    combined_stock_data = combined_stock_data.reset_index()

    # Check if the 'Date' and 'Ticker' columns exist
    if 'Date' in combined_stock_data.columns and 'Ticker' in combined_stock_data.columns:
        # Convert 'Date' column to datetime objects (if it's not already)
        if not pd.api.types.is_datetime64_any_dtype(combined_stock_data['Date']):
            try:
                # Try inferring the format for each element individually
                combined_stock_data['Date'] = pd.to_datetime(combined_stock_data['Date'], format='mixed', dayfirst=True, errors='raise')
                print("Successfully converted dates using mixed format with dayfirst=True.")


            except ValueError as e:
                print(f"Could not convert to date using format='mixed': {e}")



        # Sort the DataFrame by 'Ticker' and 'Date'
        combined_stock_data = combined_stock_data.sort_values(by=['Ticker', 'Date'])

        # Print the DataFrame info and first few rows to verify
        print("DataFrame Info:")
        print(combined_stock_data.info())  # Show DataFrame structure and data types
        print("\nFirst 5 rows (after formatting and sorting):")
        print(combined_stock_data.head())
    else:
        print("Error: The DataFrame must contain both 'Date' and 'Ticker' columns.")
else:
    print("Error: DataFrame 'combined_stock_data' not found. Make sure it is loaded first.")

Successfully converted dates using mixed format with dayfirst=True.
DataFrame Info:
<class 'pandas.core.frame.DataFrame'>
Index: 24604971 entries, 18384337 to 21194555
Data columns (total 11 columns):
 #   Column          Dtype         
---  ------          -----         
 0   level_0         int64         
 1   index           int64         
 2   Date            datetime64[ns]
 3   Low             float64       
 4   Open            float64       
 5   Volume          float64       
 6   High            float64       
 7   Close           float64       
 8   Adjusted Close  float64       
 9   Ticker          object        
 10  Index           object        
dtypes: datetime64[ns](1), float64(6), int64(2), object(2)
memory usage: 2.2+ GB
None

First 5 rows (after formatting and sorting):
           level_0     index       Date        Low       Open      Volume  \
18384337  18384337  18384337 1999-11-18  28.612303  32.546494  62546380.0   
23614454  23614454  23614454 1999-11-18  28.6

In [27]:

def populate_next_day_open(df):
    """
    Populates a new column 'Next_Day_Open' in the DataFrame with the
    'Open' value from the next day's row, grouped by 'Ticker'.

    Args:
      df: A pandas DataFrame with 'Date', 'Open', and 'Ticker' columns.

    Returns:
      A pandas DataFrame with the added 'Next_Day_Open' column.  Returns the
      original DataFrame if 'Open' or 'Ticker' is missing.
    """

    if 'Open' not in df.columns or 'Ticker' not in df.columns:
      print("Error: DataFrame must contain 'Open' and 'Ticker' columns.")
      return df # Or raise an exception, depending on desired error handling

    df['Next_Day_Open'] = df.groupby('Ticker')['Open'].shift(-1)
    return df

combined_stock_data = populate_next_day_open(combined_stock_data)


In [30]:
combined_stock_data.head()

,level_0,index,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker,Index,Next_Day_Open
18384337,18384337,18384337,1999-11-18,28.612303,32.546494,62546380.0,35.765381,31.473534,26.929760,A,/content/stock_market_data/sp500,32.546494
23614454,23614454,23614454,1999-11-18,28.612303,32.546494,62546380.0,35.765381,31.473534,27.066582,A,/content/stock_market_data/forbes2000,30.713518
18384338,18384338,18384338,1999-11-19,28.478184,30.713518,15234146.0,30.758226,28.880545,24.711119,A,/content/stock_market_data/sp500,30.713518
23614455,23614455,23614455,1999-11-19,28.478184,30.713518,15234146.0,30.758226,28.880545,24.836662,A,/content/stock_market_data/forbes2000,29.551144
18384339,18384339,18384339,1999-11-22,28.657009,29.551144,6577870.0,31.473534,31.473534,26.929760,A,/content/stock_market_data/sp500,29.551144


In [ ]:
import pandas as pd

def process_chunk(df_chunk):
    """Processes a chunk of the DataFrame: calculates market changes and filters."""

    # Check for required columns
    required_columns = ['Open', 'Close', 'Next_Day_Open', 'Ticker', 'Date']
    if not all(col in df_chunk.columns for col in required_columns):
        missing_cols = [col for col in required_columns if col not in df_chunk.columns]
        print(f"Error: DataFrame chunk must contain the following columns: {missing_cols}")
        return pd.DataFrame()  # Return an empty DataFrame if columns are missing

    # Filter out rows with 0.0 in Open, Close, or Next_Day_Open
    df_chunk_filtered = df_chunk[
        (df_chunk['Open'] != 0.0) &
        (df_chunk['Close'] != 0.0) &
        (df_chunk['Next_Day_Open'] != 0.0)
    ].copy()  # copy to prevent "SettingWithCopyWarning"

    # Calculate Market Changes (after filtering)
    df_chunk_filtered['Regular_Market_Change'] = ((df_chunk_filtered['Close'] - df_chunk_filtered['Open']) / df_chunk_filtered['Open']) * 100
    df_chunk_filtered['After_Hours_Market_Change'] = ((df_chunk_filtered['Next_Day_Open'] - df_chunk_filtered['Close']) / df_chunk_filtered['Close']) * 100

    return df_chunk_filtered

# --- Main Execution ---
# Assumes 'combined_stock_data' is already loaded and prepared

if 'combined_stock_data' in locals():
    # Define the chunk size
    chunk_size = 10000  # Adjust this based on your system's memory

    # Calculate the number of chunks
    num_chunks = (len(combined_stock_data) + chunk_size - 1) // chunk_size  # Integer division to round up

    # Initialize lists to store the top results from each chunk
    top_50_regular_list = []
    top_50_after_hours_list = []

    # Process the DataFrame in chunks
    for i in range(num_chunks):
        print(f"Processing chunk {i+1}/{num_chunks}")
        # Determine the start and end indices for the current chunk
        start_idx = i * chunk_size
        end_idx = min((i + 1) * chunk_size, len(combined_stock_data))  # Ensure we don't go past the end of the DataFrame

        # Extract the chunk from the DataFrame
        chunk = combined_stock_data.iloc[start_idx:end_idx].copy()  # Create copy of chunk

        # Process the chunk
        filtered_chunk = process_chunk(chunk)

        if not filtered_chunk.empty:  # Only proceed if the chunk wasn't empty after filtering.
            # Get the top 50 from this chunk and append to list.
            top_50_regular_list.append(filtered_chunk.nlargest(50, 'Regular_Market_Change'))
            top_50_after_hours_list.append(filtered_chunk.nlargest(50, 'After_Hours_Market_Change'))

    print("All chunks processed")

    # Combine the top 50 results from all chunks
    if top_50_regular_list:  # Check if list is not empty.
        top_50_regular = pd.concat(top_50_regular_list).nlargest(50, 'Regular_Market_Change').copy()  # Get FINAL top 50
        top_50_after_hours = pd.concat(top_50_after_hours_list).nlargest(50, 'After_Hours_Market_Change').copy()  # Get FINAL top 50

        # Print the top 50 DataFrames
        print("\nFinal Top 50 Regular Market Changes (from all chunks):")
        print(top_50_regular)

        print("\nFinal Top 50 After Hours Market Changes (from all chunks):")
        print(top_50_after_hours)
    else:
        print("No data after filtering. Check your data and filter criteria")
else:
    print("Error: DataFrame 'combined_stock_data' not found. Make sure it is loaded and prepared first.")

In [43]:
top_50_regular

,level_0,index,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker,Index,Next_Day_Open,Regular_Market_Change,After_Hours_Market_Change
24098882,24098882,24098882,2014-11-14,0.00010,0.00010,17309.0,0.325000,0.300000,0.30000,CHNGQ,/content/stock_market_data/forbes2000,0.25000,299900.019500,-16.666670
24098909,24098909,24098909,2014-12-24,0.00010,0.00010,52008.0,0.250800,0.250600,0.25060,CHNGQ,/content/stock_market_data/forbes2000,0.05000,250500.016487,-80.047886
22415625,22415625,22415625,2017-07-13,0.00010,0.00010,54370.0,0.184000,0.184000,0.18400,HERB,/content/stock_market_data/forbes2000,0.18000,183900.005006,-2.173909
1457156,1457156,1457156,2017-08-15,0.00010,0.00010,14185.0,0.060000,0.060000,0.06000,TEUFF,/content/stock_market_data/nyse,0.06000,59900.000175,0.000000
20461789,20461789,20461789,2011-09-07,0.10000,0.10000,109.0,50.000000,50.000000,50.00000,PVNC,/content/stock_market_data/forbes2000,50.00000,49899.999255,0.000000
8506926,8506926,8506926,2016-04-19,0.00010,0.00010,18743.0,0.060000,0.029900,0.02990,SPDC,/content/stock_market_data/nasdaq,0.03510,29800.000255,17.391312
6610403,6610403,6610403,2018-06-08,0.00010,0.00010,44630.0,0.030000,0.024440,0.02444,DOMR,/content/stock_market_data/nyse,0.01940,24340.000451,-20.621928
22228140,22228140,22228140,2009-09-14,1.06000,1.06000,18.0,106.349998,106.349998,83.91481,VUPPF,/content/stock_market_data/forbes2000,111.25000,9933.019266,4.607430
6610002,6610002,6610002,2016-11-02,0.00010,0.00010,5251.0,0.010000,0.010000,0.01000,DOMR,/content/stock_market_data/nyse,0.01000,9900.000029,0.000000
6610118,6610118,6610118,2017-04-21,0.00010,0.00010,15360.0,0.010000,0.010000,0.01000,DOMR,/content/stock_market_data/nyse,0.01100,9900.000029,10.000002


In [44]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=top_50_regular)

https://docs.google.com/spreadsheets/d/1wAaP9MsajWhUHEsD-LRHdVJllnKXaVO5fap9fKv55Oc#gid=0


In [45]:
top_50_after_hours

,level_0,index,Date,Low,Open,Volume,High,Close,Adjusted Close,Ticker,Index,Next_Day_Open,Regular_Market_Change,After_Hours_Market_Change
8547339,8547339,8547339,2012-02-07,0.000100,0.000100,0.0,0.000100,0.000100,0.000100,CCXI,/content/stock_market_data/nasdaq,10.490000,0.000000,1.048990e+07
20420488,20420488,20420488,2021-03-01,0.000200,0.000200,0.0,0.000200,0.000200,0.000067,CRBJF,/content/stock_market_data/forbes2000,4.780000,0.000000,2.389900e+06
16208702,16208702,16208702,2020-11-20,0.000200,0.000200,6150.0,0.000200,0.000200,0.000200,FRMC,/content/stock_market_data/sp500,1.990000,0.000000,9.949000e+05
21620064,21620064,21620064,2014-11-28,0.010000,0.010000,0.0,0.010000,0.010000,0.008714,WTBCF,/content/stock_market_data/forbes2000,69.448997,0.000000,6.943900e+05
23687580,23687580,23687580,2016-11-30,0.001000,0.001000,400.0,0.001000,0.001000,0.000834,TGOPY,/content/stock_market_data/forbes2000,4.000000,0.000000,3.999000e+05
15097432,15097432,15097432,2021-12-28,0.000100,0.000100,400.0,0.000100,0.000100,0.000100,GBIM,/content/stock_market_data/nasdaq,0.309900,0.000000,3.098000e+05
16348050,16348050,16348050,1998-10-19,6.250000,6.250000,0.0,6.250000,6.250000,6.250000,INTH,/content/stock_market_data/sp500,11250.000000,0.000000,1.799000e+05
6380244,6380244,6380244,2022-01-06,0.000100,0.000100,0.0,0.000100,0.000100,0.000100,VULC,/content/stock_market_data/nyse,0.150000,0.000000,1.499000e+05
22185624,22185624,22185624,2015-05-01,0.001000,0.001000,0.0,0.001000,0.001000,0.001000,SAHN,/content/stock_market_data/forbes2000,1.250000,0.000000,1.249000e+05
22184585,22184585,22184585,2011-03-15,0.002100,0.002100,0.0,0.002100,0.002100,0.002100,SAHN,/content/stock_market_data/forbes2000,1.800000,0.000000,8.561429e+04


In [46]:
from google.colab import sheets
sheet = sheets.InteractiveSheet(df=top_50_after_hours)

https://docs.google.com/spreadsheets/d/1dSZxDqFGE5WHCLMzaZVG9AQGnILGnlTv1iosS_B9Els#gid=0
